# JSONファイルからRAG用のベクトルDBを作る

LangChain を使って RAG を試してみた #AI - Qiita
> https://qiita.com/tinymouse/items/4d359674f6b2494bb22d

LLMアプリケーション開発のためのLangChain 後編⑤ 外部ドキュメントのロード、分割及び保存 - qiita
> https://qiita.com/utanesuke/items/6efc03eca94f7de3b9cd#json-%E3%83%AD%E3%83%BC%E3%83%80%E3%83%BC


## install

In [ ]:
!pip install unsloth langchain langchain_community langchain-huggingface sentence-transformers transformers accelerate lancedb

In [ ]:
!pip install -U langchain langchain-community lancedb

In [ ]:
!conda install 

## imports

In [1]:
## main models
# need to import unsloth 1st
from unsloth import FastLanguageModel
import transformers
import torch
from sentence_transformers import SentenceTransformer

## langchain (need to make RAG)
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import LanceDB

import lancedb
import pyarrow as pa
import json
import pprint
from pathlib import Path

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## prepare tokenizer & model

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 40960,
    load_in_4bit = True,            # 4bit uses much less memory
    load_in_8bit = False,           # A bit more accurate, uses 2x memory
)

==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.431 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## prepare pipeline

In [2]:
pipe = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    dtype=torch.float16
)

llm = HuggingFacePipeline(
    pipeline = pipe
)

NameError: name 'model' is not defined

## make TextSplitter

In [3]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=80,
    length_function=len
)

## load json datas

In [6]:
# need to avoid unicode escape & marge instruction, output
loader = JSONLoader(
    file_path="./jvn_results_merged.json",
#    file_path="./"
    jq_schema=".[] | .instruction, .output",
#    jq_schema=".[] | .title, .description"
    text_content=False
)
docs_raw = loader.load()

docs = []
for i in range(0, len(docs_raw), 2):
    inst = docs_raw[i].page_content
    out = docs_raw[i+1].page_content
    inst_meta = docs_raw[i].metadata["seq_num"]
    out_meta = docs_raw[i+1].metadata["seq_num"]

    # page_content に instruction と output をまとめる
    content = (
        "passage: Q. "
        f"{inst}  "
        "A. "
        f"{out}\n"
    )

    # メタデータとして元データを保持（任意）
    metadata = {
        "instruction": inst_meta,
        "output": out_meta
    }
    docs.append(Document(page_content=content, metadata=metadata))

pprint.pprint(docs[1])

Document(metadata={'instruction': 3, 'output': 4}, page_content='passage: Q. SQL インジェクションの脆弱性の例を教えて  A. carmelogarcia の Simple Leave Manager In PHP With Source Code における SQL インジェクションの脆弱性が発表された。A flaw has been found in code-projects Simple Leave Manager 1.0. This vulnerability affects unknown code of the file /user.php. This manipulation of the argument table causes sql injection. Remote exploitation of the attack is possible. The exploit has been published and may be used. この脆弱性はcarmelogarcia\nSimple Leave Manager In PHP With Source Code 1.0に影響を及ぼす。\n')


## prepare embedding model

In [7]:
embeddings = SentenceTransformerEmbeddings(
    model_name="./infloat_multilingual-e5-large"
)

/tmp/ipykernel_139673/2349016760.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(


## save DB && save DB

In [8]:
vectorstore = LanceDB.from_documents(
    documents=docs,
    embedding=embeddings,
    uri="./LanceDB/lance_CVE.db",
    table_name="lance_CVE",   # 無ければ自動作成
    mode="overwrite"          # 上書き作成
)

## load DB

In [10]:
db = lancedb.connect("./LanceDB/lance_CVE.db")
table = db.open_table("lance_CVE")
table.create_index(num_partitions=64)

In [ ]:
db = lancedb.connect("./LanceDB/rules.lancedb")
table = db.open_table("rules")
table.create_index(num_partitions=64)

## search DB

In [11]:
base = """
あなたは与えられたソースコードから脆弱性を発見するホワイトハッカーです。
回答の際は以下の3つを行いなさい。
1. 脆弱性の指摘
2. 放置することによるリスクの提示
3. 脆弱性の解消方法の提示

出力の際は以下の3つに注意しなさい。
1. 日本語で出力
2. 明快かつ簡潔に
3. マークダウン形式を使用

また出力の際にはコンテキストの内容を元に生成しなさい
"""

question = """
以下は100面ダイスを振るアプリのソースコードです。
セクションハイジャックの脆弱性が存在しますか？
"""

source = """
---
<?php
// php_d100_roller.php
// Simple 100-sided dice roller application (single file)
// Usage: Place this file on your web server and open it in a browser.
// For local testing: run `php -S localhost:8000` and visit http://localhost:8000/php_d100_roller.php

session_start();

// Keep roll history in session (max 100 entries)
if (!isset($_SESSION['d100_history'])) {
    $_SESSION['d100_history'] = [];
}

$errors = [];
$results = [];
$total = 0;
$count = 1;

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // Get roll count (clamped between 1–100)
    $count = isset($_POST['count']) ? intval($_POST['count']) : 1;
    if ($count < 1) $count = 1;
    if ($count > 100) $count = 100;

    // Optional label
    $label = isset($_POST['label']) ? trim($_POST['label']) : '';

    // Perform rolls
    for ($i = 0; $i < $count; $i++) {
        $roll = random_int(1, 100);
        $results[] = $roll;
        $total += $roll;
    }

    // Add to history (newest first)
    $entry = [
        'time' => date('Y-m-d H:i:s'),
        'count' => $count,
        'label' => $label,
        'results' => $results,
        'total' => $total,
    ];

    array_unshift($_SESSION['d100_history'], $entry);
    if (count($_SESSION['d100_history']) > 100) {
        $_SESSION['d100_history'] = array_slice($_SESSION['d100_history'], 0, 100);
    }
}

$history = $_SESSION['d100_history'];
?>
<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>100-sided Dice Roller</title>
<style>
    body { font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; padding: 20px; }
    .box { max-width: 820px; margin: 0 auto; }
    input[type="number"] { width: 80px; }
    .badge { display:inline-block; padding:6px 10px; margin:4px; border-radius:6px; background:#eee; }
    .roll { font-weight:700; }
    .history { margin-top:20px; }
    .card { padding:12px; border:1px solid #ddd; border-radius:8px; margin-bottom:12px; }
    .muted { color:#666; font-size:0.9rem; }
</style>
</head>
<body>
<div class="box">
    <h1>100-sided Dice Roller</h1>
    <form method="post" action="<?php echo htmlspecialchars($_SERVER['PHP_SELF']); ?>">
    <label>Number of rolls (1–100): <input type="number" name="count" value="<?php echo htmlspecialchars($count); ?>" min="1" max="100"></label>
    &nbsp;
    <label>Label (optional): <input type="text" name="label" value=""></label>
    &nbsp;
    <button type="submit">Roll</button>
    </form>

    <?php if (!empty($results)): ?>
    <div class="card">
        <div class="muted">Timestamp: <?php echo htmlspecialchars($entry['time']); ?></div>
        <h2>Results</h2>
        <div>
        <?php foreach ($results as $i => $r): ?>
            <span class="badge roll">#<?php echo $i+1; ?>: <?php echo $r; ?></span>
        <?php endforeach; ?>
        </div>
        <p>Total: <strong><?php echo $total; ?></strong> / Average: <strong><?php echo count($results) ? round($total / count($results), 2) : 0; ?></strong></p>
        <?php if ($entry['label'] !== ''): ?><p>Label: <?php echo htmlspecialchars($entry['label']); ?></p><?php endif; ?>
    </div>
    <?php endif; ?>

    <div class="history">
    <h2>History (last <?php echo count($history); ?> rolls)</h2>
    <?php if (empty($history)): ?>
        <p class="muted">No rolls yet.</p>
    <?php else: ?>
        <?php foreach ($history as $idx => $h): ?>
        <div class="card">
            <div class="muted"><?php echo htmlspecialchars($h['time']); ?> — Rolls: <?php echo $h['count']; ?><?php if ($h['label'] !== ''): ?> — Label: <?php echo htmlspecialchars($h['label']); ?><?php endif; ?></div>
            <div style="margin-top:8px;">
            <?php foreach ($h['results'] as $i => $r): ?>
                <span class="badge"><?php echo $r; ?></span>
            <?php endforeach; ?>
            </div>
            <p style="margin-top:8px;">Total: <?php echo $h['total']; ?> / Average: <?php echo count($h['results']) ? round($h['total'] / count($h['results']), 2) : 0; ?></p>
        </div>
        <?php endforeach; ?>
    <?php endif; ?>
    </div>

    <div style="margin-top:20px;" class="muted">
    <p>Note: Uses <code>random_int(1, 100)</code> for cryptographically secure random number generation. The session keeps up to 100 history entries.</p>
    </div>
</div>
</body>
</html>
"""

content_base = base+question+source

In [12]:
cvec = embeddings.embed_query("search_query: "+question)
docs_sim = (
    table.search(cvec)
         .limit(5)
         .to_list()
)

# vectorstore.similarity_search(query=content_base, k=5)
print(docs_sim[0]["text"])

passage: Q. 動的に操作されるコードリソースの不適切な制御に関する脆弱性の例を教えて  A. TerraMaster TOS における動的に操作されるコードリソースの不適切な制御に関する脆弱性が存在する。 この問題を受けるアプリのバージョンはTerraMaster
TOS 4.1.29 未満である。



## add context for prompt

In [13]:
# make message from template
context = ""
for loop in range(len(docs_sim)):
    context = context+(f"context_{loop+1}. {docs_sim[loop]["text"]}")

messages = [{"role" : "user", "content" : content_base +"\n---\ncontext: "+ context}]

print(messages)

[{'role': 'user', 'content': '\nあなたは与えられたソースコードから脆弱性を発見するホワイトハッカーです。\n回答の際は以下の3つを行いなさい。\n1. 脆弱性の指摘\n2. 放置することによるリスクの提示\n3. 脆弱性の解消方法の提示\n\n出力の際は以下の3つに注意しなさい。\n1. 日本語で出力\n2. 明快かつ簡潔に\n3. マークダウン形式を使用\n\nまた出力の際にはコンテキストの内容を元に生成しなさい\n\n以下は100面ダイスを振るアプリのソースコードです。\nセクションハイジャックの脆弱性が存在しますか？\n\n---\n<?php\n// php_d100_roller.php\n// Simple 100-sided dice roller application (single file)\n// Usage: Place this file on your web server and open it in a browser.\n// For local testing: run `php -S localhost:8000` and visit http://localhost:8000/php_d100_roller.php\n\nsession_start();\n\n// Keep roll history in session (max 100 entries)\nif (!isset($_SESSION[\'d100_history\'])) {\n    $_SESSION[\'d100_history\'] = [];\n}\n\n$errors = [];\n$results = [];\n$total = 0;\n$count = 1;\n\nif ($_SERVER[\'REQUEST_METHOD\'] === \'POST\') {\n    // Get roll count (clamped between 1–100)\n    $count = isset($_POST[\'count\']) ? intval($_POST[\'count\']) : 1;\n    if ($count < 1) $count = 1;\n    if ($count > 100) $c

In [14]:
# infer
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# セクションハイジャックの脆弱性の指摘

## 1. 脆弱性の指摘  
このコードには**セクションハイジャック**の脆弱性は存在**しません**。  
セクションハイジャックとは、セッションIDや認証トークンが不適切に管理されることで、不正にセッションを乗っ取られる脆弱性を指しますが、このコードではセッションの管理に`$_SESSION`を使用しており、セキュリティ的な問題は見られません。

## 2. 放置することによるリスクの提示  
セクションハイジャックのリスクが存在しないので、この脆弱性の放置によるリスクは発生しません。

## 3. 脆弱性の解消方法の提示  
このコードではセッションハイジャックの脆弱性が存在しないため、解消方法は不要です。ただし、セッションの安全性を高めるために以下のような対策を検討することを推奨します：

- セッションIDの暗号化や、セッションクッキーに`HttpOnly`や`Secure`属性を設定する。
- セッションの有効期限を設定し、長時間のセッション維持を避ける。<|im_end|>


## another dataset version

In [ ]:
# -*- coding: utf-8 -*-
from unsloth import FastLanguageModel
import transformers
import torch

import json
from pathlib import Path
from typing import List, Dict, Any
import os
import math

import pyarrow as pa
import lancedb
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import time

# ----------------------------
# Config
# ----------------------------
EMBED_MODEL = "./infloat_multilingual-e5-large/"
DB_PATH = "./LanceDB/rules.lancedb"
TABLE_NAME = "rules"
JSON_PATH = Path("./rspec_rules.json")
BATCH_SIZE = 256
USE_DEVICE = "cuda"

# モデルのロード
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 40960,
    load_in_4bit = False,
    load_in_8bit = False,
)

pipe = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    dtype=torch.float16
)

# LangChainのHuggingFacePipelineは非推奨の警告が出ることがあるため、
# 直接呼び出すか、必要な場合のみインポートしてください。
# ここではRAG部分で直接 generate を呼んでいるため llm 変数は必須ではありませんが、
# 念のため残す場合は transformers の pipeline をラップします。
# from langchain_huggingface import HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline=pipe)

# ----------------------------
# 1) Embedding モデル
# ----------------------------
emb_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": USE_DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)

def embed(text: str) -> List[float]:
    """E5 埋め込みを取得"""
    formatted = f"passage: {text}"
    emb = emb_model.embed_query(formatted)
    arr = np.asarray(emb, dtype=np.float32)
    return arr.tolist()

# ----------------------------
# 2) Splitter
# ----------------------------
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)

# ----------------------------
# 3) LanceDB 接続 & schema 作成
# ----------------------------
db = lancedb.connect(DB_PATH)

if TABLE_NAME in db.table_names():
    print(f"テーブル {TABLE_NAME} が存在するため削除します...")
    db.drop_table(TABLE_NAME)

print("確認: 埋め込み次元を取得しています...")
sample_dim = len(embed("this is a sample"))
print("Embedding dim detected:", sample_dim)
EMB_DIM = sample_dim

# 【修正点1】 list_size=EMB_DIM を追加して固定長リストにする
schema = pa.schema([
    ("id", pa.string()),
    ("title", pa.string()),
    ("chunk", pa.string()),
    ("embedding", pa.list_(pa.float32(), list_size=EMB_DIM)), 
    ("url", pa.string()),
])

table = db.create_table(TABLE_NAME, schema=schema, mode="overwrite")
print(f"テーブル {TABLE_NAME} を作成しました。")

# ----------------------------
# 4) JSON 読み込みとデータ投入
# ----------------------------
if JSON_PATH.exists():
    with JSON_PATH.open("r", encoding="utf-8") as f:
        data = json.load(f)

    rows_to_insert = []
    
    print("データ処理を開始します...")
    for item in data:
        title = item["title"]
        description = item["description"]
        url = item.get("url", "")

        base_text = f"{title}\n\n{description}"
        chunks = splitter.split_text(base_text)

        for idx, ch in enumerate(chunks):
            if ch.strip() == title:
                continue
            
            emb = embed(ch)

            rows_to_insert.append({
                "id": f"{url}#{idx}",
                "title": title,
                "chunk": ch,
                "embedding": emb,
                "url": url
            })
            
            # 【修正点2】 メモリ節約のため一定数たまったらinsertしてリストをクリアする
            if len(rows_to_insert) >= BATCH_SIZE:
                table.add(rows_to_insert)
                print(f"✓ {len(rows_to_insert)} 件追加しました")
                rows_to_insert = [] # リセット

    # ループ終了後に残っているデータを追加
    if rows_to_insert:
        table.add(rows_to_insert)
        print(f"✓ 残りの {len(rows_to_insert)} 件を追加しました")

else:
    print(f"警告: {JSON_PATH} が見つかりません。ダミーデータなしで検索に進みます。")

# ----------------------------
# 5) 検索関数
# ----------------------------
def search(query: str, k: int = 5):
    formatted = f"query: {query}"
    query_emb = emb_model.embed_query(formatted)

    results = (
        table.search(query_emb)
        .metric("cosine")
        .limit(k)
        .to_list()
    )
    return results

# ----------------------------
# 6) コンテキスト生成と RAG
# ----------------------------
def build_context(docs: List[Dict[str, Any]]):
    parts = []
    for i, d in enumerate(docs):
        title = d.get("title", "")
        chunk = d.get("chunk", "")
        # LanceDBのバージョンによっては _distance だったり score だったりします
        score = d.get("_score", d.get("_distance", 0.0))
        parts.append(f"[{i+1}] (score={score:.4f}) {title}\n{chunk}")
    return "\n\n----\n\n".join(parts)

def rag_answer(question: str, code: str, k: int = 5):
    docs = search(question, k=k)
    
    # 検索結果デバッグ用
    # for d in docs:
    #     print(f"Found: {d['title']}")

    if not docs:
        print("関連ドキュメントが見つかりませんでした。")
        return

    context = build_context(docs)
    
    # プロンプト内の変数を適切に埋め込み
    system_prompt = f"""You are a white hat hacker tasked with discovering vulnerabilities in the provided source code.
Perform the following three actions on the source code below:
1. Identify the vulnerability
2. Present the risks of leaving it unaddressed
3. Provide a solution to eliminate the vulnerability

Ensure your output adheres to the following three points:
1. Output in Japanese
2. Be clear and concise
3. Use Markdown format
"""

    user_message = f"""Answer based on the following context.
{context}

Question: {question}
---
{code}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True,
    )

    from transformers import TextStreamer
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    _ = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 4096,
        temperature = 0.6,
        top_p = 0.95,
        top_k = 20,
        streamer = streamer,
    )

# ----------------------------
# 7) 実行
# ----------------------------
if __name__ == "__main__":
    q = "Does the following code contain an XSS vulnerability?"
    c = """
<?php
// php_d100_roller.php
// Simple 100-sided dice roller application (single file)
// Usage: Place this file on your web server and open it in a browser.
// For local testing: run `php -S localhost:8000` and visit http://localhost:8000/php_d100_roller.php

session_start();

// Keep roll history in session (max 100 entries)
if (!isset($_SESSION['d100_history'])) {
    $_SESSION['d100_history'] = [];
}

$errors = [];
$results = [];
$total = 0;
$count = 1;

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // Get roll count (clamped between 1–100)
    $count = isset($_POST['count']) ? intval($_POST['count']) : 1;
    if ($count < 1) $count = 1;
    if ($count > 100) $count = 100;

    // Optional label
    $label = isset($_POST['label']) ? trim($_POST['label']) : '';

    // Perform rolls
    for ($i = 0; $i < $count; $i++) {
        $roll = random_int(1, 100);
        $results[] = $roll;
        $total += $roll;
    }

    // Add to history (newest first)
    $entry = [
        'time' => date('Y-m-d H:i:s'),
        'count' => $count,
        'label' => $label,
        'results' => $results,
        'total' => $total,
    ];

    array_unshift($_SESSION['d100_history'], $entry);
    if (count($_SESSION['d100_history']) > 100) {
        $_SESSION['d100_history'] = array_slice($_SESSION['d100_history'], 0, 100);
    }
}

$history = $_SESSION['d100_history'];
?>
<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>100-sided Dice Roller</title>
<style>
    body { font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; padding: 20px; }
    .box { max-width: 820px; margin: 0 auto; }
    input[type="number"] { width: 80px; }
    .badge { display:inline-block; padding:6px 10px; margin:4px; border-radius:6px; background:#eee; }
    .roll { font-weight:700; }
    .history { margin-top:20px; }
    .card { padding:12px; border:1px solid #ddd; border-radius:8px; margin-bottom:12px; }
    .muted { color:#666; font-size:0.9rem; }
</style>
</head>
<body>
<div class="box">
    <h1>100-sided Dice Roller</h1>
    <form method="post" action="<?php echo htmlspecialchars($_SERVER['PHP_SELF']); ?>">
    <label>Number of rolls (1–100): <input type="number" name="count" value="<?php echo htmlspecialchars($count); ?>" min="1" max="100"></label>
    &nbsp;
    <label>Label (optional): <input type="text" name="label" value=""></label>
    &nbsp;
    <button type="submit">Roll</button>
    </form>

    <?php if (!empty($results)): ?>
    <div class="card">
        <div class="muted">Timestamp: <?php echo htmlspecialchars($entry['time']); ?></div>
        <h2>Results</h2>
        <div>
        <?php foreach ($results as $i => $r): ?>
            <span class="badge roll">#<?php echo $i+1; ?>: <?php echo $r; ?></span>
        <?php endforeach; ?>
        </div>
        <p>Total: <strong><?php echo $total; ?></strong> / Average: <strong><?php echo count($results) ? round($total / count($results), 2) : 0; ?></strong></p>
        <?php if ($entry['label'] !== ''): ?><p>Label: <?php echo htmlspecialchars($entry['label']); ?></p><?php endif; ?>
    </div>
    <?php endif; ?>

    <div class="history">
    <h2>History (last <?php echo count($history); ?> rolls)</h2>
    <?php if (empty($history)): ?>
        <p class="muted">No rolls yet.</p>
    <?php else: ?>
        <?php foreach ($history as $idx => $h): ?>
        <div class="card">
            <div class="muted"><?php echo htmlspecialchars($h['time']); ?> — Rolls: <?php echo $h['count']; ?><?php if ($h['label'] !== ''): ?> — Label: <?php echo htmlspecialchars($h['label']); ?><?php endif; ?></div>
            <div style="margin-top:8px;">
            <?php foreach ($h['results'] as $i => $r): ?>
                <span class="badge"><?php echo $r; ?></span>
            <?php endforeach; ?>
            </div>
            <p style="margin-top:8px;">Total: <?php echo $h['total']; ?> / Average: <?php echo count($h['results']) ? round($h['total'] / count($h['results']), 2) : 0; ?></p>
        </div>
        <?php endforeach; ?>
    <?php endif; ?>
    </div>

    <div style="margin-top:20px;" class="muted">
    <p>Note: Uses <code>random_int(1, 100)</code> for cryptographically secure random number generation. The session keeps up to 100 history entries.</p>
    </div>
</div>
</body>
</html>
"""
    print("\nQUESTION:", q)
    print("\n=== ANSWER ===\n")
    rag_answer(q, c, k=3)


## use created vectorDB ver

In [2]:
# -*- coding: utf-8 -*-
from unsloth import FastLanguageModel
import transformers
import torch

import json
from pathlib import Path
from typing import List, Dict, Any
import os
import math

import pyarrow as pa
import lancedb
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import time

# ----------------------------
# Config
# ----------------------------
EMBED_MODEL = "./infloat_multilingual-e5-large/"
DB_PATH = "./LanceDB/rules.lancedb"
TABLE_NAME = "rules"
JSON_PATH = Path("./rspec_rules.json")
BATCH_SIZE = 256
USE_DEVICE = "cuda"

# モデルのロード
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 40960,
    load_in_4bit = True,
    load_in_8bit = False,
)

pipe = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    dtype=torch.float16
)

# LangChainのHuggingFacePipelineは非推奨の警告が出ることがあるため、
# 直接呼び出すか、必要な場合のみインポートしてください。
# ここではRAG部分で直接 generate を呼んでいるため llm 変数は必須ではありませんが、
# 念のため残す場合は transformers の pipeline をラップします。
# from langchain_huggingface import HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline=pipe)

# ----------------------------
# 1) Embedding モデル
# ----------------------------
emb_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": USE_DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)

def embed(text: str) -> List[float]:
    """E5 埋め込みを取得"""
    formatted = f"passage: {text}"
    emb = emb_model.embed_query(formatted)
    arr = np.asarray(emb, dtype=np.float32)
    return arr.tolist()

# ----------------------------
# 2) Splitter
# ----------------------------
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)

# ----------------------------
# 3) LanceDB 接続 & schema 作成
# ----------------------------
db = lancedb.connect(DB_PATH)

# すでにテーブルがあるか確認
if TABLE_NAME in db.table_names():
    print(f"テーブル {TABLE_NAME} は既に存在します。ロードします...")
    table = db.open_table(TABLE_NAME)
else:
    print(f"テーブル {TABLE_NAME} が存在しないため、新規作成します...")

    print("確認: 埋め込み次元を取得しています...")
    sample_dim = len(embed("this is a sample"))
    print("Embedding dim detected:", sample_dim)
    EMB_DIM = sample_dim

    schema = pa.schema([
        ("id", pa.string()),
        ("title", pa.string()),
        ("chunk", pa.string()),
        ("embedding", pa.list_(pa.float32(), list_size=EMB_DIM)), 
        ("url", pa.string()),
    ])

    table = db.create_table(TABLE_NAME, schema=schema, mode="overwrite")
    print(f"テーブル {TABLE_NAME} を作成しました。")

# ----------------------------
# 4) JSON 読み込みとデータ投入
# ----------------------------
if JSON_PATH.exists():
    with JSON_PATH.open("r", encoding="utf-8") as f:
        data = json.load(f)

    rows_to_insert = []
    
    print("データ処理を開始します...")
    for item in data:
        title = item["title"]
        description = item["description"]
        url = item.get("url", "")

        base_text = f"{title}\n\n{description}"
        chunks = splitter.split_text(base_text)

        for idx, ch in enumerate(chunks):
            if ch.strip() == title:
                continue
            
            emb = embed(ch)

            rows_to_insert.append({
                "id": f"{url}#{idx}",
                "title": title,
                "chunk": ch,
                "embedding": emb,
                "url": url
            })
            
            # 【修正点2】 メモリ節約のため一定数たまったらinsertしてリストをクリアする
            if len(rows_to_insert) >= BATCH_SIZE:
                table.add(rows_to_insert)
                print(f"✓ {len(rows_to_insert)} 件追加しました")
                rows_to_insert = [] # リセット

    # ループ終了後に残っているデータを追加
    if rows_to_insert:
        table.add(rows_to_insert)
        print(f"✓ 残りの {len(rows_to_insert)} 件を追加しました")

else:
    print(f"警告: {JSON_PATH} が見つかりません。ダミーデータなしで検索に進みます。")

# ----------------------------
# 5) 検索関数
# ----------------------------
def search(query: str, k: int = 5):
    formatted = f"query: {query}"
    query_emb = emb_model.embed_query(formatted)

    results = (
        table.search(query_emb)
        .metric("cosine")
        .limit(k)
        .to_list()
    )
    return results

# ----------------------------
# 6) コンテキスト生成と RAG
# ----------------------------
def build_context(docs: List[Dict[str, Any]]):
    parts = []
    for i, d in enumerate(docs):
        title = d.get("title", "")
        chunk = d.get("chunk", "")
        # LanceDBのバージョンによっては _distance だったり score だったりします
        score = d.get("_score", d.get("_distance", 0.0))
        parts.append(f"[{i+1}] (score={score:.4f}) {title}\n{chunk}")
    return "\n\n----\n\n".join(parts)

def rag_answer(question: str, code: str, k: int = 5):
    docs = search(question, k=k)
    
    # 検索結果デバッグ用
    # for d in docs:
    #     print(f"Found: {d['title']}")

    if not docs:
        print("関連ドキュメントが見つかりませんでした。")
        return

    context = build_context(docs)
    
    # プロンプト内の変数を適切に埋め込み
    system_prompt = f"""You are a white hat hacker tasked with discovering vulnerabilities in the provided source code.
Perform the following three actions on the source code below:
1. Identify the vulnerability
2. Present the risks of leaving it unaddressed
3. Provide a solution to eliminate the vulnerability

Ensure your output adheres to the following three points:
1. Output in Japanese
2. Be clear and concise
3. Use Markdown format
"""

    user_message = f"""Answer based on the following context.
{context}

Question: {question}
---
{code}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True,
    )

    from transformers import TextStreamer
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    _ = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 4096,
        temperature = 0.6,
        top_p = 0.95,
        top_k = 20,
        streamer = streamer,
    )

# ----------------------------
# 7) 実行
# ----------------------------
if __name__ == "__main__":
    q = "Does the following code contain a section hijacking vulnerability?"
    c = """
<?php
// php_d100_roller.php
// Simple 100-sided dice roller application (single file)
// Usage: Place this file on your web server and open it in a browser.
// For local testing: run `php -S localhost:8000` and visit http://localhost:8000/php_d100_roller.php

session_start();

// Keep roll history in session (max 100 entries)
if (!isset($_SESSION['d100_history'])) {
    $_SESSION['d100_history'] = [];
}

$errors = [];
$results = [];
$total = 0;
$count = 1;

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // Get roll count (clamped between 1–100)
    $count = isset($_POST['count']) ? intval($_POST['count']) : 1;
    if ($count < 1) $count = 1;
    if ($count > 100) $count = 100;

    // Optional label
    $label = isset($_POST['label']) ? trim($_POST['label']) : '';

    // Perform rolls
    for ($i = 0; $i < $count; $i++) {
        $roll = random_int(1, 100);
        $results[] = $roll;
        $total += $roll;
    }

    // Add to history (newest first)
    $entry = [
        'time' => date('Y-m-d H:i:s'),
        'count' => $count,
        'label' => $label,
        'results' => $results,
        'total' => $total,
    ];

    array_unshift($_SESSION['d100_history'], $entry);
    if (count($_SESSION['d100_history']) > 100) {
        $_SESSION['d100_history'] = array_slice($_SESSION['d100_history'], 0, 100);
    }
}

$history = $_SESSION['d100_history'];
?>
<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>100-sided Dice Roller</title>
<style>
    body { font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; padding: 20px; }
    .box { max-width: 820px; margin: 0 auto; }
    input[type="number"] { width: 80px; }
    .badge { display:inline-block; padding:6px 10px; margin:4px; border-radius:6px; background:#eee; }
    .roll { font-weight:700; }
    .history { margin-top:20px; }
    .card { padding:12px; border:1px solid #ddd; border-radius:8px; margin-bottom:12px; }
    .muted { color:#666; font-size:0.9rem; }
</style>
</head>
<body>
<div class="box">
    <h1>100-sided Dice Roller</h1>
    <form method="post" action="<?php echo htmlspecialchars($_SERVER['PHP_SELF']); ?>">
    <label>Number of rolls (1–100): <input type="number" name="count" value="<?php echo htmlspecialchars($count); ?>" min="1" max="100"></label>
    &nbsp;
    <label>Label (optional): <input type="text" name="label" value=""></label>
    &nbsp;
    <button type="submit">Roll</button>
    </form>

    <?php if (!empty($results)): ?>
    <div class="card">
        <div class="muted">Timestamp: <?php echo htmlspecialchars($entry['time']); ?></div>
        <h2>Results</h2>
        <div>
        <?php foreach ($results as $i => $r): ?>
            <span class="badge roll">#<?php echo $i+1; ?>: <?php echo $r; ?></span>
        <?php endforeach; ?>
        </div>
        <p>Total: <strong><?php echo $total; ?></strong> / Average: <strong><?php echo count($results) ? round($total / count($results), 2) : 0; ?></strong></p>
        <?php if ($entry['label'] !== ''): ?><p>Label: <?php echo htmlspecialchars($entry['label']); ?></p><?php endif; ?>
    </div>
    <?php endif; ?>

    <div class="history">
    <h2>History (last <?php echo count($history); ?> rolls)</h2>
    <?php if (empty($history)): ?>
        <p class="muted">No rolls yet.</p>
    <?php else: ?>
        <?php foreach ($history as $idx => $h): ?>
        <div class="card">
            <div class="muted"><?php echo htmlspecialchars($h['time']); ?> — Rolls: <?php echo $h['count']; ?><?php if ($h['label'] !== ''): ?> — Label: <?php echo htmlspecialchars($h['label']); ?><?php endif; ?></div>
            <div style="margin-top:8px;">
            <?php foreach ($h['results'] as $i => $r): ?>
                <span class="badge"><?php echo $r; ?></span>
            <?php endforeach; ?>
            </div>
            <p style="margin-top:8px;">Total: <?php echo $h['total']; ?> / Average: <?php echo count($h['results']) ? round($h['total'] / count($h['results']), 2) : 0; ?></p>
        </div>
        <?php endforeach; ?>
    <?php endif; ?>
    </div>

    <div style="margin-top:20px;" class="muted">
    <p>Note: Uses <code>random_int(1, 100)</code> for cryptographically secure random number generation. The session keeps up to 100 history entries.</p>
    </div>
</div>
</body>
</html>
"""
    print("\nQUESTION:", q)
    print("\n=== ANSWER ===\n")
    rag_answer(q, c, k=3)


==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.431 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


テーブル rules は既に存在します。ロードします...


RuntimeError: lance error: LanceError(IO): does not have sufficient data, len: 0, bytes: b"", /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/lance-io-0.39.0/src/utils.rs:136:13